In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

In [13]:
# system lib
import sklearn
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score,recall_score
from sklearn.svm import SVC, LinearSVC, NuSVC

from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier  #随机森林
from sklearn import tree

In [14]:
from time import time
import datetime
import numpy as np

In [4]:
#读取数据
data = pd.read_csv('preprocess_train.csv')

In [5]:
# 查看数据
data.head()

,sample_id,feature0,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,...,feature98,feature99,feature100,feature101,feature102,feature103,feature104,feature105,feature106,label
0,0,51.567250,288358.4,NaN,1.855900,201.460169,6.582261,-0.516321,5.636771e+11,222221.210940,...,31459.628135,254.582034,0.0,-2.014506e+08,159.299350,0.603211,NaN,-5.256075,180.977310,4
1,1,63.804874,288358.4,1.106802,1.050387,391.605375,13.323439,4.662871,NaN,-144247.357554,...,42830.526855,270.580779,0.0,-1.534970e+09,NaN,0.506220,0.552654,16.505952,314.783263,5
2,2,49.138527,288358.4,1.111649,0.767127,130.708067,6.485547,5.696815,5.474603e+11,-428840.326950,...,249963.241809,160.207067,0.0,7.998345e+08,112.632639,0.080100,0.235920,64.707581,183.304610,2
3,3,NaN,288358.4,1.109169,NaN,258.521076,0.716737,23.238461,-3.539891e+11,NaN,...,125478.297201,196.223295,0.0,-9.246920e+07,138.431470,2.548783,1.414810,-9.662399,212.302670,2
4,4,76.520831,288358.4,1.113410,5.795408,256.038997,-1.803483,14.040495,-1.071014e+11,649972.266570,...,186266.423019,179.083883,0.0,-6.796351e+08,259.858740,0.337643,0.228832,59.733069,135.541233,2


In [20]:
# 用平均值填补缺失值
data = data.fillna(data.mean())

In [21]:
# 查看数据  
data.head()

,feature0,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,...,feature98,feature99,feature100,feature101,feature102,feature103,feature104,feature105,feature106,label
0,51.567250,288358.4,1.132154,1.855900,201.460169,6.582261,-0.516321,5.636771e+11,222221.210940,2.002797e+14,...,31459.628135,254.582034,0.0,-2.014506e+08,159.299350,0.603211,1.482161,-5.256075,180.977310,4
1,63.804874,288358.4,1.106802,1.050387,391.605375,13.323439,4.662871,8.634842e+10,-144247.357554,2.601894e+15,...,42830.526855,270.580779,0.0,-1.534970e+09,189.389530,0.506220,0.552654,16.505952,314.783263,5
2,49.138527,288358.4,1.111649,0.767127,130.708067,6.485547,5.696815,5.474603e+11,-428840.326950,1.363373e+14,...,249963.241809,160.207067,0.0,7.998345e+08,112.632639,0.080100,0.235920,64.707581,183.304610,2
3,63.748003,288358.4,1.109169,1.177755,258.521076,0.716737,23.238461,-3.539891e+11,82388.389996,-1.338632e+15,...,125478.297201,196.223295,0.0,-9.246920e+07,138.431470,2.548783,1.414810,-9.662399,212.302670,2
4,76.520831,288358.4,1.113410,5.795408,256.038997,-1.803483,14.040495,-1.071014e+11,649972.266570,2.800250e+15,...,186266.423019,179.083883,0.0,-6.796351e+08,259.858740,0.337643,0.228832,59.733069,135.541233,2


In [25]:
def load_data(filename):
    """根据数据格式，读取数据中的X和分类标签y
    """

    # 读取数据
    data = pd.read_csv(filename)
    x_data = data.iloc[:, 1:]
    ylabel = data.iloc[:, 0]

    return x_data, ylabel

In [22]:
def evaluate_classifier( real_label_list,predict_label_list):
    """
       return Precision, Recall and ConfusionMatrix
       Input : predict_label_list,real_label_list
    """
    msg=''
    Confusion_matrix = confusion_matrix( real_label_list,predict_label_list)
    msg += '\n Confusion Matrix\n ' + str(Confusion_matrix)
    precision = precision_score(real_label_list,predict_label_list, average=None)
    recall = recall_score(real_label_list,predict_label_list, average=None)
    msg += '\n Precision of tag 0 and 1 =%s' %str(precision)
    msg += '\n Recall of tag 0 and 1 =%s' %str(recall)

    return msg

In [34]:
from sklearn.metrics import auc, roc_curve
from sklearn.model_selection import GridSearchCV


def test_svm(train_file, test_file):
    """用SVM分类 """
    # use SVM directly

    train_xdata, train_ylabel = load_data(train_file)

    test_xdata, test_ylabel = load_data(test_file)

    print('\nuse SVM directly')

    #classifier1 = SVC(kernel='linear')
    #classifier1 = SVC(kernel='linear',probability=True, C=200, cache_size=500)
    classifier1 = SVC(kernel='linear',probability=True,C=10, cache_size=500)

    print(train_xdata)
    print(train_ylabel)
    classifier1.fit(train_xdata, train_ylabel)

    predict_labels = classifier1.predict(test_xdata)
    accuracy = accuracy_score(test_ylabel, predict_labels)
    print("\n The Classifier's Accuracy is : %f" %accuracy)
    #
    eval_msg = evaluate_classifier(test_ylabel,predict_labels)
    print(eval_msg)
    #
    #GridSearchCV搜索最优参数示例
    print("GridSearchCV搜索最优参数......")
    t0 = time()
    param_grid = {
        "C": [1e3, 5e3, 1e4, 5e4, 1e5],
        "gamma": [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1],
    }
    classifier1 = GridSearchCV(SVC(kernel="rbf", class_weight="balanced",probability=True), param_grid)
    classifier1 = classifier1.fit(train_xdata, train_ylabel)
    print("done in %0.3fs" % (time() - t0))
    print("Best estimator found by grid search:")
    print(classifier1.best_estimator_)


    #对于SVM来说，概率是通过交叉验证得到的，与其预测的结果未必一致，对小数据集来说，此概率没什么意义
    probas_ = classifier1.predict_proba(test_xdata)

    #对于二分类问题，可为分类器绘制ROC曲线，计算AUC
    # Compute ROC curve and area the curve
    fpr, tpr, thresholds = roc_curve(test_ylabel, probas_[:, 1])
    roc_auc = auc(fpr, tpr)
    print("Area under the ROC curve : %f" % roc_auc)

    # Plot ROC curve
    plt.clf()
    plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('%s SVM ROC' %train_file)
    plt.legend(loc="lower right")
    plt.show()

In [30]:
# 把文件按照8:2的比例分为训练集和测试集
train, test = train_test_split(data, test_size=0.2)

# 保存训练集和测试集
train.to_csv('train.csv', index=False)
test.to_csv('test.csv', index=False)

# 查看训练集和测试集‘
train.head()

,feature0,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,...,feature98,feature99,feature100,feature101,feature102,feature103,feature104,feature105,feature106,label
4829,63.748003,288358.400000,2.110507,1.177755,-23.106416,8.117357,7.672728,-1.733279e+11,3.087299e+05,6.983706e+14,...,1.856221e+04,255.331953,0.0,9.528981e+07,230.092537,1.154086,0.581878,12.941307,208.421173,1
4361,47.204543,288358.400000,1.102553,0.251668,288.654997,6.649990,-12.971772,9.144012e+10,8.858028e+04,-2.637882e+14,...,1.406383e+05,205.054301,0.0,1.666794e+08,145.555906,0.280151,0.043296,2.321539,144.938765,0
1645,75.931289,288358.400000,1.105570,0.668918,176.177699,19.977496,6.131551,-2.698272e+11,8.238839e+04,-2.508767e+15,...,9.824399e+04,239.531745,0.0,-4.941773e+06,153.254757,3.858668,0.764019,2.577558,201.685074,0
5655,63.748003,288358.400000,1.132154,0.124467,251.501641,19.387243,-10.005651,2.810544e+11,-8.801847e+05,-2.562260e+14,...,7.782960e+04,153.684939,0.0,1.150051e+08,165.741397,1.390493,0.611307,8.752637,229.718098,0
2180,27.167679,-362477.378321,-1.483620,1.177755,362.452515,28.504287,112.350655,5.530028e+12,4.425724e+06,-4.358516e+15,...,1.190535e+06,992.450855,0.0,6.043904e+08,202.266194,4.005191,12.974987,112.939245,21.475839,0


In [29]:
test.head()

,feature0,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,...,feature98,feature99,feature100,feature101,feature102,feature103,feature104,feature105,feature106,label
1841,47.577167,288358.4,1.103240,0.021721,309.199252,15.421998,5.717172,4.055096e+10,-2.281506e+04,-1.083166e+15,...,190842.189118,177.809956,0.0,-4.433264e+08,256.397308,0.129623,0.370015,-22.857353,180.049759,0
2462,67.590857,288358.4,1.103676,0.032363,107.354807,9.879153,-2.168596,4.690212e+11,-4.286419e+05,-4.435898e+15,...,13038.316385,207.992086,0.0,-6.780954e+07,129.445684,1.390493,2.952191,-9.801221,229.718098,3
5704,76.456461,288358.4,1.100418,0.385087,195.757143,20.210396,5.835945,-4.666464e+10,-1.146079e+06,-1.416351e+15,...,214362.436117,145.376685,0.0,6.853442e+08,105.961754,0.691853,1.134088,-12.928667,341.182442,0
2662,56.754214,288358.4,1.106366,0.322526,262.858368,6.238443,7.417670,2.198837e+11,-6.212794e+04,-3.577306e+15,...,96259.711789,189.993620,0.0,-4.432442e+08,218.890049,0.238269,1.482161,16.707478,193.733091,2
6022,54.425629,288358.4,1.103828,1.177755,249.857318,2.992818,20.167333,7.315501e+10,-1.251532e+05,5.636279e+15,...,147171.921968,195.074296,0.0,8.950201e+08,104.598571,3.082780,0.031112,29.835077,182.784486,1


In [33]:
# 查看label项的内容有哪些
train['label'].unique()
test['label'].unique()

array([0, 5, 2, 4, 3, 1])

In [46]:
data_train=load_data('train.csv')
print(data_train[0].shape)

(5036, 107)


In [49]:
# 打印最后一列的内容
data_train[0].iloc[:, -1]


0       1
1       0
2       0
3       0
4       0
       ..
5031    2
5032    0
5033    0
5034    0
5035    4
Name: label, Length: 5036, dtype: int64

In [50]:
# 将最后一列设为ylabel
train_ylabel = data_train[0].iloc[:, -1]
# 其他的设为xdata
train_xdata = data_train[0].iloc[:, :-1]

In [2]:
# 最后一列为ylable,其他为xdata
test_xdata, test_ylabel = load_data('test.csv')

print('\nuse SVM directly')

#classifier1 = SVC(kernel='linear')
#classifier1 = SVC(kernel='linear',probability=True, C=200, cache_size=500)

classifier1 = SVC(kernel='linear',probability=True,C=10, cache_size=1024)


NameError: name 'load_data' is not defined

In [39]:
print(train_xdata)

           feature1  feature2  feature3    feature4   feature5    feature6  \
0     288358.400000  2.110507  1.177755  -23.106416   8.117357    7.672728   
1     288358.400000  1.102553  0.251668  288.654997   6.649990  -12.971772   
2     288358.400000  1.105570  0.668918  176.177699  19.977496    6.131551   
3     288358.400000  1.132154  0.124467  251.501641  19.387243  -10.005651   
4    -362477.378321 -1.483620  1.177755  362.452515  28.504287  112.350655   
...             ...       ...       ...         ...        ...         ...   
5031  288358.400000  1.132154  1.640445  322.951084  15.728056    0.290246   
5032  285239.586221 -1.797589  1.177755   79.383552   8.942381   -0.293853   
5033  288358.400000  1.110953  0.472234  225.724825  22.061659   17.042899   
5034  288358.400000  1.105097  0.666487  256.805811   6.512474   43.704738   
5035  288358.400000  1.132154  0.396618  323.606865   7.754419   -0.525821   

          feature7      feature8      feature9  feature10  ... 